## 0. Configure paths and URLs

In [ ]:
import os

from shutil import copyfile


# URL to IXI images
ixi_images_path = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T1.tar'

# URL to IXI labels
ixi_labels_path = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI.xls'

# Root path where images should be stored
ixi_folder = os.path.join(os.path.expanduser('~'), 'data', 'IXI')

# Path to MNI152 template
mni_template = os.path.join('/', 'usr', 'local', 'fsl', 'data', 'standard', 
                            'MNI152lin_T1_1mm_brain.nii.gz')

## 1. Setup folder structure and ensure all necessary software and packages is installed

In [ ]:
from shutil import which


if not os.path.isdir(ixi_folder):
    os.mkdir(ixi_folder)
    
tar_path = os.path.join(ixi_folder, 'IXI-T1.tar')
xls_path = os.path.join(ixi_folder, 'IXI.xls')

def exists(software: str):
    return which(software) is not None

software = ['recon-all', 'mri_convert', 'fslreorient2std', 'flirt']
missing = [name for name in software if not exists(name)]

if len(missing) > 0:
    raise ValueError(f'Missing required software {missing}')

## 2. Download, extract and reorganize the data

In [ ]:
import tarfile
import numpy as np
import pandas as pd

from pyment.utils import download

download(ixi_images_path, tar_path)
download(ixi_labels_path, xls_path)

# Unpack the tar file
tar = tarfile.open(tar_path)
tar.extractall(os.path.join(ixi_folder, 'raw', 'images'))
tar.close()

# Reorganize the labels into a CSV
df = pd.read_excel(xls_path)
filenames = os.listdir(os.path.join(ixi_folder, 'raw', 'images'))

# Match filenames with label ids
mapping = {int(filename[3:6]): filename.split('.')[0] for filename in filenames}
df['id'] = df['IXI_ID'].apply(lambda x: mapping[x] if x in mapping else np.nan)

# Simplify sex coding
df['sex'] = df['SEX_ID (1=m, 2=f)'].apply(lambda x: {1: 'M', 2: 'F'}[x])
df['age'] = df['AGE']

# Simplify table and drop duplicates and rows missing data
df = df[['id', 'sex', 'age']]
df = df.drop_duplicates('id')
df = df.dropna(subset=['id', 'sex', 'age'])

df.to_csv(os.path.join(ixi_folder, 'raw', 'labels.csv'), index=False)

## 3. Preprocess the data

In [ ]:
from pyment.utils.preprocessing import autorecon1_folder

if not os.path.isdir(os.path.join(ixi_folder, 'recon')):
    os.mkdir(os.path.join(ixi_folder, 'recon'))

autorecon1_folder(os.path.join(ixi_folder, 'raw', 'images'), 
                  os.path.join(ixi_folder, 'recon'), threads=4)

In [ ]:
if not os.path.isdir(os.path.join(ixi_folder, 'brainmasks', 'images')):
    os.makedirs(os.path.join(ixi_folder, 'brainmasks', 'images'))

for subject in os.listdir(os.path.join(ixi_folder, 'recon')):
    brainmask = os.path.join(ixi_folder, 'recon', subject, 'mri', 'brainmask.mgz')
    brainmask = os.path.abspath(brainmask)
    
    if not os.path.isfile(brainmask):
        print(f'Skipping {subject}. Missing brainmask')
        continue
        
    target = os.path.join(ixi_folder, 'brainmasks', 'images', f'{subject}.mgz')
        
    # Check if the symlink already exists, and if so overwrite it
    if os.path.exists(target) or os.path.islink(target):
        os.remove(target)
        
    os.symlink(brainmask, target)
    
initial = len(os.listdir(os.path.join(ixi_folder, 'recon')))
final = len(os.listdir(os.path.join(ixi_folder, 'brainmasks', 'images')))
failed = initial - final

f'Finished brain mask extraction step. {failed} images failed'

In [ ]:
from pyment.utils.preprocessing import convert_mgz_to_nii_gz_folder


if not os.path.isdir(os.path.join(ixi_folder, 'nifti', 'images')):
    os.makedirs(os.path.join(ixi_folder, 'nifti', 'images'))

convert_mgz_to_nii_gz_folder(os.path.join(ixi_folder, 'brainmasks', 'images'),
                             os.path.join(ixi_folder, 'nifti', 'images'))

initial = len(os.listdir(os.path.join(ixi_folder, 'brainmasks', 'images')))
final = len(os.listdir(os.path.join(ixi_folder, 'nifti', 'images')))
failed = initial - final

f'Finished conversion step. {failed} images failed'

In [ ]:
from pyment.utils.preprocessing import reorient2std_folder


if not os.path.isdir(os.path.join(ixi_folder, 'reoriented', 'images')):
    os.makedirs(os.path.join(ixi_folder, 'reoriented', 'images'))
    
reorient2std_folder(os.path.join(ixi_folder, 'nifti', 'images'),
                    os.path.join(ixi_folder, 'reoriented', 'images'))

initial = len(os.listdir(os.path.join(ixi_folder, 'nifti', 'images')))
final = len(os.listdir(os.path.join(ixi_folder, 'reoriented', 'images')))
failed = initial - final

f'Finished reorientation step. {failed} images failed'

In [ ]:
from pyment.utils.preprocessing import flirt_folder


if not os.path.isdir(os.path.join(ixi_folder, 'mni152', 'images')):
    os.makedirs(os.path.join(ixi_folder, 'mni152', 'images'))
    
flirt_folder(os.path.join(ixi_folder, 'reoriented', 'images'),
             os.path.join(ixi_folder, 'mni152', 'images'), template=mni_template)

initial = len(os.listdir(os.path.join(ixi_folder, 'reoriented', 'images')))
final = len(os.listdir(os.path.join(ixi_folder, 'mni152', 'images')))
failed = initial - final

f'Finished alignment step. {failed} images failed'

In [ ]:
from pyment.utils.preprocessing import crop_folder


if not os.path.isdir(os.path.join(ixi_folder, 'cropped', 'images')):
    os.makedirs(os.path.join(ixi_folder, 'cropped', 'images'))
    
bounds = ((6, 173), (2, 214), (0, 160))
    
crop_folder(os.path.join(ixi_folder, 'mni152', 'images'),
            os.path.join(ixi_folder, 'cropped', 'images'), bounds)

initial = len(os.listdir(os.path.join(ixi_folder, 'mni152', 'images')))
final = len(os.listdir(os.path.join(ixi_folder, 'cropped', 'images')))
failed = initial - final

f'Finished cropping step. {failed} images failed'

In [ ]:
copyfile(os.path.join(ixi_folder, 'raw', 'labels.csv'),
         os.path.join(ixi_folder, 'cropped', 'labels.csv'))

files = os.listdir(os.path.join(ixi_folder, 'cropped', 'images'))

f'Finished preprocessing {len(files)} IXI images'